In [1]:
import pandas as pd
import cv2
import numpy as np
import scipy
import time

import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator
%matplotlib inline

source = 0
deg_to_rad = np.pi/180
spat_freq = 0.08

In [2]:
import os

im_dim = 720
video_dir = "/home/marco/phd/wp1/venv_blender/render_center/"

files = sorted(
    [os.path.join(video_dir, f) for f in os.listdir(video_dir) if os.path.isfile(os.path.join(video_dir, f))]
)

video = np.zeros((im_dim, im_dim, len(files)), dtype=np.uint8)

for idx, f in enumerate(files):
    im = cv2.imread(f, cv2.IMREAD_GRAYSCALE)
    video[:,:,idx] = im

In [3]:
import re

kernel_dir = "/home/marco/phd/wp1/data_allen/gabor_filters_3d/"

files =\
    [os.path.join(kernel_dir, f) for f in os.listdir(kernel_dir) if os.path.isfile(os.path.join(kernel_dir, f))]

gs = {}

for f in files:
    n=re.findall(r'\d+', f)[-1]
    gs[f"k{n}"] = scipy.io.loadmat(f)[f"k{n}"]

In [4]:
video_short = video[:,:,:100]

In [5]:
video_short.shape

(720, 720, 100)

In [7]:
video_conv = scipy.signal.fftconvolve(
    video_short,
    gs["k1"],
    mode='same',
)

video_conv = video_conv - np.min(video_conv)
video_conv = np.divide(video_conv, np.max(video_conv))

In [20]:
video_conv_ml = scipy.io.loadmat("/home/marco/phd/wp1/video_conv/video_conv_norm1.mat")["video_conv_norm"]

In [21]:
video_conv.shape, video_conv_ml.shape

((720, 720, 100), (720, 720, 100))

In [27]:
err_mask = np.logical_not(np.isclose(video_conv, video_conv_ml))

In [28]:
err = np.abs(video_conv[err_mask] - video_conv_ml[err_mask])

In [32]:
np.sum(err) / (video_conv.shape[0]*video_conv.shape[1]*video_conv.shape[2])*100

0.8488366011503384

In [29]:
np.mean(err)

0.008495032252090093

In [30]:
np.max(err)

0.060485799080101765